In [1]:
#import all libraries
import pandas as pd
import requests
import bs4
import re
import logging
import concurrent.futures
import threading
import math
from time import sleep
from bs4 import BeautifulSoup
import numpy as np
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
import json

header section

In [2]:
headers={
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/117.0',
    'Accept':'text/html, */*; q=0.01',
    'Accept-Language':'en-US,en;q=0.5',
    'X-Requested-With':'XMLHttpRequest'
    }

url section

In [3]:
iran_ketab_url='https://www.iranketab.ir'
all_books_url='/book'

In [4]:
def call_for_page(page_number, timeout):
        response=requests.post(url='https://www.iranketab.ir/book' ,headers=headers,json={'pagenumber':str(page_number),'pagesize':20},timeout=timeout)
        return response

In [5]:
def get_a_tags_from_iranKetab(total_pages_number:int,pages_arr):
        
        out=[]               #output of threads-thread's result
        result_list=[]       #list of all book's atag
        count_of_request=100  #number of requests sent to the website at each iteration 
        
        #maximum number of requests(iterations) because maybe we have limited time, i do not use it here also
        max_number_request=math.ceil((total_pages_number*2)/count_of_request)
        count = 0
        workers=40  #number of threads
        sleep_time=1

        # pbar = tqdm(total=count_first_keyword_list, desc="Request to get suggestion titles of job")
        # while (len(pages_arr)) and (count <= max_number_request):
        while (len(pages_arr)):
                sleep(sleep_time)
                #update req_list
                if len(pages_arr) > count_of_request:
                    req_list =pages_arr[:count_of_request].copy()
                else:
                    req_list = pages_arr.copy()

                with ThreadPoolExecutor(max_workers=workers) as executor:
                    future_to_url = (executor.submit(call_for_page, pg_number, 30) for pg_number in req_list)
                    for future in as_completed(future_to_url):
                        try:
                            data = future.result()
                            out.append(data)
                        except Exception as exc:
                            continue

                for item in out:
                    if item.status_code == 200:
                        this_page_number=int(json.loads(item.request.body)['pagenumber'])
                        if this_page_number in req_list:
                            print('page '+str(this_page_number)+' html added to result')
                            result_list.extend(BeautifulSoup(item.content,'html.parser').select('div.col-lg-6 > div > div > div > h4 > a '))
                            pages_arr=np.delete(pages_arr,np.where(pages_arr==this_page_number))

                # count += 1
                # pbar.update(count_last_keyword_list - len(keyword_list))
        return result_list,pages_arr

In [6]:
def get_num_pages(url):
    response=requests.post(url=url,headers=headers,json={'pagenumber':1,'pagesize':20})
    soup=BeautifulSoup(response.content,'html.parser')
    total_pages=int(re.split('\\=|\\&',soup.
                                        select('.PagedList-skipToLast > a:nth-child(1)')[0].\
                                                                                           get('url'))[1])
    return total_pages

In [7]:
def find_book_links(total_pages_num):
     pages_arr=np.arange(1,total_pages_num+1)
     all_tags,not_scraped_pages=get_a_tags_from_iranKetab(total_pages_num,pages_arr)
     while len(not_scraped_pages)!=0:
        new_a_tags,not_scraped_pages=get_a_tags_from_iranKetab(total_pages_num,not_scraped_pages)
        all_tags.extend(new_a_tags)
     all_tags=list(set(all_tags))
     return all_tags

In [8]:
def save_into_csv(book_links) :
    import csv
    with open('books_url.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['link'])
            writer.writerows([[iran_ketab_url+item] for item in book_links])

In [9]:
def get_book_link():
    total_pages_num=get_num_pages(iran_ketab_url+all_books_url)
    book_tags=find_book_links(total_pages_num)
    book_links=[]
    for a_tag in book_tags:
        book_links.append(a_tag.get('href'))
    save_into_csv(book_links)
    

In [10]:
get_book_link()

page 1 html added to result
page 12 html added to result
page 5 html added to result
page 19 html added to result
page 3 html added to result
page 17 html added to result
page 4 html added to result
page 7 html added to result
page 10 html added to result
page 6 html added to result
page 11 html added to result
page 20 html added to result
page 36 html added to result
page 28 html added to result
page 26 html added to result
page 37 html added to result
page 32 html added to result
page 30 html added to result
page 45 html added to result
page 42 html added to result
page 34 html added to result
page 41 html added to result
page 46 html added to result
page 15 html added to result
page 16 html added to result
page 2 html added to result
page 14 html added to result
page 25 html added to result
page 18 html added to result
page 21 html added to result
page 9 html added to result
page 43 html added to result
page 29 html added to result
page 8 html added to result
page 22 html added to r